## Datos geograficos

Vamos a trabajar con algunos datos geograficos, en este caso usaremos el dataset delitos_2021. Este contiene datos de delitos cometidos en la Ciudad de Buenos Aires, se pueden encontrar en la pagina oficial de datos de la Ciudad de Buenos Aires https://data.buenosaires.gob.ar/dataset/delitos.

Inspeccionemos el archivo

In [5]:
import pandas as pd

delitos_2022 = pd.read_excel(r'C:\Users\usuario\Documents\Programming\Databases\delitos_2022.xlsx')
delitos_2023 = pd.read_excel(r'C:\Users\usuario\Documents\Programming\Databases\delitos_2023.xlsx')

In [13]:
#¿Cuantas y cuales son las columnas del dataset?
delitos_2022.info()

print("El dataset tiene", len(delitos_2022.columns), "columnas")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140918 entries, 0 to 140917
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id-mapa   140918 non-null  int64         
 1   anio      140918 non-null  int64         
 2   mes       140918 non-null  object        
 3   dia       140918 non-null  object        
 4   fecha     140918 non-null  datetime64[ns]
 5   franja    140871 non-null  float64       
 6   tipo      140918 non-null  object        
 7   subtipo   140918 non-null  object        
 8   uso_arma  140918 non-null  object        
 9   uso_moto  140918 non-null  object        
 10  barrio    138527 non-null  object        
 11  comuna    138525 non-null  float64       
 12  latitud   138533 non-null  float64       
 13  longitud  138533 non-null  float64       
 14  cantidad  140918 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 16.1+ MB
El datas

In [14]:
#¿Cuantos valores en total hay en 2022 y en 2023?
print("Cantidad de delitos en 2022: ", len(delitos_2022))
print("Cantidad de delitos en 2023: ", len(delitos_2023))

140918

In [17]:
#¿Cuantos y cuales tipos de delitos hay?
tipo_delitos = pd.unique(delitos_2022["tipo"])
cantidad_delitos = len(tipo_delitos)
print("Tipos de delitos: ", tipo_delitos)
print("Cantidad de delitos: ", cantidad_delitos)

Tipos de delitos:  ['Robo' 'Hurto' 'Vialidad' 'Homicidios' 'Lesiones' 'Amenazas']
Cantidad de delitos:  6


In [ ]:


#¿Cuantos y cuales subtipos de delitos hay?


#¿Cuantos y cuales uso de arma hay?



In [ ]:
#Veamos cuantos homicidios hubieron en 2022 y en 2023


In [ ]:
#¿Podriamos cuantificar el aumento de homicidios del 2022 al 2023?



In [ ]:
#Veamos homicidios dolosos en 2022 en la comuna 1


In [ ]:
#Cuales son los Subtipos de vialidad?


Queremos trabajar con datos geograficos pero los datos de latitud y longitud estan en otro formato y ademas hay datos faltantes que valen 0, es importante corregir o hacer algo con esto, el proximo modulo nos consentraremos en la limpieza y preparacion de datos, igualmente aqui vemos un ejemplo de como ir arreglando un dataset

In [ ]:
#Corregimos el valor de latitud y longitud dividiendo hasta que quede bien
def normalizar_coord(valor):
    """
    Divide repetidamente entre 10 hasta que el valor tenga dos dígitos enteros.
    Respeta el signo original.
    """
    if pd.isna(valor):
        return None

    v = float(valor)
    signo = 1 if v >= 0 else -1
    v = abs(v)

    # dividir mientras la parte entera tenga más de 2 dígitos
    while int(v) >= 100:
        v /= 10

    return signo * v

delitos_2022['longitud']=delitos_2022['longitud'].apply(normalizar_coord)
delitos_2023['longitud']=delitos_2023['longitud'].apply(normalizar_coord)
delitos_2022['latitud']=delitos_2022['latitud'].apply(normalizar_coord)
delitos_2023['latitud']=delitos_2023['latitud'].apply(normalizar_coord)


#Transformamos el valor 0 a None
delitos_2022.replace(0, None,inplace=True)
delitos_2023.replace(0, None,inplace=True)

In [ ]:
delitos_2022.latitud.hist(bins=100)
plt.yscale('log')

#Veamos los datos que estan mal
delitos_2022[delitos_2022.latitud>-34] #Son  3 amenazas decidimos borrarlas
delitos_2022=delitos_2022[delitos_2022.latitud<-34]


In [ ]:
delitos_2022.longitud.hist(bins=100)
plt.yscale('log')

#parece estar mal el signo, lo corregimos
indice=delitos_2022[delitos_2022['longitud']>0].index
delitos_2022.loc[indice,'longitud']=delitos_2022.loc[indice,'longitud']*-1

In [ ]:
delitos_2023.latitud.hist(bins=100)
plt.yscale('log')


Primero que todo instalen geopandas y otras herramientas utiles.

In [ ]:
!pip install geopandas
!pip install unzip
!pip install wget
!pip install contextily

El archivo de los datos geoespaciales de CABA los podemos conseguir de [aquí](https://data.buenosaires.gob.ar/dataset/comunas/resource/Juqdkmgo-612222-resource). Para hacerlo más fácil, en la celda de abajo lo descargamos y descomprimimos desde python (sino, lo pueden hacer ustedes, solo necesitamos los archivos "comunas_wgs84.shp" y "comunas_wgs84.shx"). **Esto lo hace en el directorio actual en el que estamos ubicados**.

In [ ]:
# Busco el zip file
!wget https://cdn.buenosaires.gob.ar/datosabiertos/datasets/comunas/comunas-zip.zip
# Lo descomprimo
!unzip "comunas-zip.zip"

Ahora importamos el módulo (y matplotlib), y usamos los archivos que descargamos crear un `GeoDataFrame` y graficar el mapa de las comunas de CABA.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# leer datos geoespaciales con GeoPandas, estos vienen en formato de shapefiles (.shp)
# Shapefiles: geospatial ArcGIS data have mandatory and optional files.
# mandatory file extensions for a shapefile are .shp, .shx and .dbf.
# optional files are: .prj, .xml, .sbn and .sbx
# https://gisgeography.com/arcgis-shapefile-files-types-extensions/
#
# # por ejemplo para CABA:
# https://data.buenosaires.gob.ar/dataset/comunas/resource/Juqdkmgo-612222-resource
# los archivos comunas_wgs84.shp y comunas_wgs84.shx ya están en el drive
comunas = gpd.read_file('comunas_wgs84.shp')
# (aunque se especifique uno solo, GeoPandas busca tambien el .shx!!!)


fig, ax = plt.subplots(figsize=(5,5))
comunas.plot(ax=ax)

Que tal si ponemos un mapa de las calles de fondo mejor, usemos contextily OpenStreetMap para eso!

In [ ]:
import contextily as ctx

# --- 1. Convertir GeoDataFrames al CRS de contextily (EPSG:3857) ---
comunas_3857 = comunas.to_crs(epsg=3857)

# --- 3. Crear figura y graficar ---
fig, ax = plt.subplots(figsize=(10,10))

# Comunas primero
comunas_3857.boundary.plot(ax=ax, color="blue", linewidth=1.2)

# --- 4. Agregar mapa base DEPUÉS de plotear los datos ---
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=13)

# --- 6. Estética ---
ax.set_axis_off()
plt.legend(prop={'size': 12})
plt.title("Delitos en CABA con calles (OpenStreetMap)", fontsize=14)
plt.show()



Ahora tratemos de ver los delitos del 2022 sobre este mapa

In [ ]:
from shapely.geometry import Point, Polygon
from pyproj import Transformer
# specify our CRS (coordinate reference system)
# EPSG 4326 corresponde al sistema convencional WGS84
# empleado para la representación de la cartografía a nivel mundial
# https://epsg.io/4326
crs = {'init': 'epsg:4326'}

# Especificamos la geometria, convirtiendo longitud y latitud en shapely Points
geometry = [Point(xy) for xy in zip( delitos_2022["longitud"], delitos_2022["latitud"])]

# Creamos el GeoDataFrame
geo_df_2022 = gpd.GeoDataFrame(delitos_2022, crs = crs, geometry = geometry)

# --- 1. Convertir GeoDataFrames al CRS de contextily (EPSG:3857) ---
comunas_3857 = comunas.to_crs(epsg=3857)
geo_df_2022 = geo_df_2022.to_crs(epsg=3857)

# --- 2. Convertir tus límites de lon/lat a EPSG:3857 ---
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
xmin, ymin = transformer.transform(-58.550, -34.725)
xmax, ymax = transformer.transform(-58.300, -34.500)

# --- 3. Crear figura y graficar ---
fig, ax = plt.subplots(figsize=(10,10))

# Comunas primero
comunas_3857.boundary.plot(ax=ax, color="blue", linewidth=1.2)

# Puntos de delitos
#geo_df_2022[geo_df_2022["tipo"]=="Robo"].plot(ax=ax, markersize=1, color="green", marker=".", label="Robo")
geo_df_2022[geo_df_2022["tipo"]=="Homicidios"].plot(ax=ax, markersize=5, color="red", marker="x", label="Homicidio")

# --- 4. Agregar mapa base DEPUÉS de plotear los datos ---
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=13)

# --- 5. Ajustar límites ---
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# --- 6. Estética ---
ax.set_axis_off()
plt.legend(prop={'size': 12})
plt.title("Delitos en CABA con calles (OpenStreetMap)", fontsize=14)
plt.show()



Ahora grafiquemos el mapa de homicidios en capital federal para 2022 y 2023

In [ ]:
from shapely.geometry import Point, Polygon
from pyproj import Transformer
def safe_point(x, y):
    if pd.isna(x) or pd.isna(y):
        return None
    return Point(x, y)
# specify our CRS (coordinate reference system)
# EPSG 4326 corresponde al sistema convencional WGS84
# empleado para la representación de la cartografía a nivel mundial
# https://epsg.io/4326
crs = {'init': 'epsg:4326'}


# Creamos el GeoDataFrame
geometry = [safe_point(x, y) for x,y in zip( delitos_2022["longitud"], delitos_2022["latitud"])]
geo_df_2022 = gpd.GeoDataFrame(delitos_2022, crs = crs, geometry = geometry)
geometry = [safe_point(x, y) for x,y in zip( delitos_2023["longitud"], delitos_2023["latitud"])]
geo_df_2023 = gpd.GeoDataFrame(delitos_2023, crs = crs, geometry = geometry)

# --- 1. Convertir GeoDataFrames al CRS de contextily (EPSG:3857) ---
comunas_3857 = comunas.to_crs(epsg=3857)
geo_df_2022 = geo_df_2022.to_crs(epsg=3857)
geo_df_2023 = geo_df_2023.to_crs(epsg=3857)

# --- 2. Convertir tus límites de lon/lat a EPSG:3857 ---
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
xmin, ymin = transformer.transform(-58.550, -34.725)
xmax, ymax = transformer.transform(-58.300, -34.500)

# --- 3. Crear figura y graficar ---
fig, ax = plt.subplots(figsize=(10,10))

# Comunas primero
comunas_3857.boundary.plot(ax=ax, color="blue", linewidth=1.2)

# Puntos de delitos
geo_df_2022[geo_df_2022["tipo"]=="Homicidios"].plot(ax=ax, markersize=10, color="red", marker="x", label="Homicidio 2022")
geo_df_2023[geo_df_2023["tipo"]=="Homicidios"].plot(ax=ax, markersize=10, color="green", marker="x", label="Homicidio 2023")

# --- 4. Agregar mapa base DEPUÉS de plotear los datos ---
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=13)

# --- 5. Ajustar límites ---
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# --- 6. Estética ---
ax.set_axis_off()
plt.legend(prop={'size': 12})
plt.title("Delitos en CABA con calles (OpenStreetMap)", fontsize=14)
plt.show()


### Actividad

1) Grafique nuevamente la robos y homicidios del 2022 pero coloque una leyenda con el numero total de eventos de cada uno.

2) Grafique en otro Vialidad separando los dos subtipos para 2022.

3) Grafique los homicidios del 2022 y 2023 separando ahora en Femicidio y Homicidios dolosos, con dos colores distintos.

#Clase 6

Ahora veamos el informe preliminar del delito 2023, traten como tarea, replicar alguno de los graficos o analisis, veran que ya son capases de realizar algo asi!

### Planetas y el uso de escala logaritmica

Ahora vamos a trabajar con la distancia y los periodos orbitales de los planetas del sistema solar.


1) Plotear los períodos vs las distancias de los planetas como puntos y en una escala doble logarítmica (log X, log Y). Estan viendo la tercela ley de Keppler! $T^{2}\propto a^{3}$
2) Escribir el nombre del planeta cerca del punto correspondiente a ese planeta en el plot (puntos adicionales si el texto no se superpone y se lee claro, más puntos adicionales si usan flechas)
3) Trazar dos lineas punteadas (una vertical, una horizontal) que se crucen en el punto de la Tierra en el gráfico.

In [ ]:
import numpy as np

# Distancia planeta-sol medido en terminos de la distancia tierra-sol
distancias = np.array([58.3,  107.7,  149.6,  227.4,  777.9, 1427.1, 2875.9, 4499.1])


#Periodo orbital en años
periodos = np.array([0.24, 0.62, 1.00, 1.88, 11.86, 29.46, 84.01, 164.8])

#Nombre de los planetas
planetas = ["Mercurio", "Venus", "Tierra", "Marte", "Jupiter", "Saturno", "Urano", "Neptuno"]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(periodos,distancias,'-*')
plt.ylabel('Periodo [años]')
plt.xlabel('Distancia [millones de km]')
plt.yscale('log')
plt.xscale('log')

# log(T^2) = 2*log(T)

In [ ]:
#Linealicemos!

## A seguir aplicando:

Ya hemos visto muchos tipos de graficos y aun existen muchos mas pero no son tan importantes como los que vimos hasta ahora. Asi que para ganar mas intuición y compreder cuando usar cada tipo de grafico, hagamos mas analisis de datasets.

### Casos COVID

Esta vez vamos a trabajar con los datos de los casos confirmados de Covid en la provincia de Santa Fe, que están en el archivo [`'casos_covid_confirmados_santafe.csv'`](https://drive.google.com/file/d/1DvshOSsVU34KdhCe1k85wyi-KVI9g6QS/view?usp=sharing). Los mismos fueron obtenidos reduciendo el dataset oficial de los casos de covid en todo el país.

1. Cargue los contenidos del csv en un DataFrame de Pandas y explore el dataset. En particular, pedimos reportar cuántos registros contiene y qué columnas tiene. ¿Cuáles son todos los valores posibles de la columna `"clasificacion"`?
2. Calcular cuánta gente de Santa Fe falleció por covid. ¿Qué porcentaje de los casos totales terminaron en muertes (o sea, cuál es la mortalidad)? Haga un grafico.
3. Calcule cuántos casos y muertes hubo en cada departamento de la provincia de Santa Fe. Haga un grafico.
4. Haga un histograma de edades de los casos confirmados. Ojo, hay que descartar primero los que tienen la edad contada en meses (mire la columna `"edad_años_meses"`). Haga también un segundo histograma de edades solo de los fallecidos. *(Sí, en el dataset hay alguien que figura que tiene 222 años... esos errores pasan. Se puede intentar filtrar esos casos para que no molesten en el histograma.)*
5. Haga el gráfico de casos diarios de covid. Para eso utilizá la columna `"fecha_apertura"`.


In [ ]:
import pandas as pd

covid_df = pd.read_csv('casos_covid_confirmados_santafe.csv')

In [ ]:
#Hagamos grafico de tortas
#covid_df['fallecido'].value_counts().plot.pie()

In [ ]:
#.size()

### Cambio climatico

Bajar los datos de temperaturas globales promedio del aire, de este sitio: https://raw.githubusercontent.com/sbu-python-summer/python-tutorial/master/day-4/nasa-giss.txt

(los datos son de la NASA: https://data.giss.nasa.gov/gistemp/graphs/)

Hay 3 columnas con datos: el año, el cambio de temperatura (No-smoothing = no suavizado), y una representación suavizada del cambio de temperatura (Lowess).

1) Leer estos datos usando Pandas – no son csv, son de ancho fijo (fwf),  Tip: ver pd.read_fwf()
2) Plotear la representación suavizada del cambio de temperatura como una línea
3) Plotear los datos de cambio de temperatura como puntos. Colorear los puntos en azul si son negativos (< 0) y rojos si son positivos (>=0).

In [ ]:
import pandas as pd

#Ayuda de carga de los datos
url = 'https://raw.githubusercontent.com/sbu-python-summer/python-tutorial/master/day-4/nasa-giss.txt'
# leemos el archivo directamente con pandas
# fwf = fixed-width format (columnas delimitadas por espaciamiento fijo)
df = pd.read_fwf(url, index_col=0, skiprows=8, header=None)   # index_col = 0 hace que tengamos dicha columna como índice
                                                              #skip_rows ignora ese número de filas antes de los datos
                                                              #header= None evita que use la primera fila de datos como nombres de las columnas

#Se puede renombrar las columnas
df.rename(columns={1: "Temperatura", 2: "Suavizado"}, inplace=True) #renombro las columnas en la misma variable con inplace=True
